In [301]:
from pathlib import Path
import pandas as pd
import tensorflow as tf
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [817]:
file_path = Path("archive/movies.csv")
df_movie = pd.read_csv(file_path)

In [818]:
df_movie["released"] = df_movie["released"].str.split("\s+\(").str[0]
#df_movie["released_country"] = df_movie["released"].str.split("\s+\(").str[1]
df_movie["released"] = pd.to_datetime(df_movie['released'])


In [819]:
df_movie ["released"] = df_movie["released"].dt.month_name()

In [820]:
df_movie.head()

,name,rating,genre,year,released,score,votes,director,writer,star,country,budget,gross,company,runtime
0,The Shining,R,Drama,1980,June,8.4,927000.0,Stanley Kubrick,Stephen King,Jack Nicholson,United Kingdom,19000000.0,46998772.0,Warner Bros.,146.0
1,The Blue Lagoon,R,Adventure,1980,July,5.8,65000.0,Randal Kleiser,Henry De Vere Stacpoole,Brooke Shields,United States,4500000.0,58853106.0,Columbia Pictures,104.0
2,Star Wars: Episode V - The Empire Strikes Back,PG,Action,1980,June,8.7,1200000.0,Irvin Kershner,Leigh Brackett,Mark Hamill,United States,18000000.0,538375067.0,Lucasfilm,124.0
3,Airplane!,PG,Comedy,1980,July,7.7,221000.0,Jim Abrahams,Jim Abrahams,Robert Hays,United States,3500000.0,83453539.0,Paramount Pictures,88.0
4,Caddyshack,R,Comedy,1980,July,7.3,108000.0,Harold Ramis,Brian Doyle-Murray,Chevy Chase,United States,6000000.0,39846344.0,Orion Pictures,98.0


In [821]:
df_movie = df_movie.drop(columns=['score', 'votes'])

In [822]:
df_movie.head()

,name,rating,genre,year,released,director,writer,star,country,budget,gross,company,runtime
0,The Shining,R,Drama,1980,June,Stanley Kubrick,Stephen King,Jack Nicholson,United Kingdom,19000000.0,46998772.0,Warner Bros.,146.0
1,The Blue Lagoon,R,Adventure,1980,July,Randal Kleiser,Henry De Vere Stacpoole,Brooke Shields,United States,4500000.0,58853106.0,Columbia Pictures,104.0
2,Star Wars: Episode V - The Empire Strikes Back,PG,Action,1980,June,Irvin Kershner,Leigh Brackett,Mark Hamill,United States,18000000.0,538375067.0,Lucasfilm,124.0
3,Airplane!,PG,Comedy,1980,July,Jim Abrahams,Jim Abrahams,Robert Hays,United States,3500000.0,83453539.0,Paramount Pictures,88.0
4,Caddyshack,R,Comedy,1980,July,Harold Ramis,Brian Doyle-Murray,Chevy Chase,United States,6000000.0,39846344.0,Orion Pictures,98.0


In [823]:
df_movie['trixbudget']= df_movie['gross'] - 3*df_movie['budget']

In [824]:
df_movie = df_movie.dropna()

In [825]:
df_movie.head()

,name,rating,genre,year,released,director,writer,star,country,budget,gross,company,runtime,trixbudget
0,The Shining,R,Drama,1980,June,Stanley Kubrick,Stephen King,Jack Nicholson,United Kingdom,19000000.0,46998772.0,Warner Bros.,146.0,-10001228.0
1,The Blue Lagoon,R,Adventure,1980,July,Randal Kleiser,Henry De Vere Stacpoole,Brooke Shields,United States,4500000.0,58853106.0,Columbia Pictures,104.0,45353106.0
2,Star Wars: Episode V - The Empire Strikes Back,PG,Action,1980,June,Irvin Kershner,Leigh Brackett,Mark Hamill,United States,18000000.0,538375067.0,Lucasfilm,124.0,484375067.0
3,Airplane!,PG,Comedy,1980,July,Jim Abrahams,Jim Abrahams,Robert Hays,United States,3500000.0,83453539.0,Paramount Pictures,88.0,72953539.0
4,Caddyshack,R,Comedy,1980,July,Harold Ramis,Brian Doyle-Murray,Chevy Chase,United States,6000000.0,39846344.0,Orion Pictures,98.0,21846344.0


In [826]:
df_movie['success'] = df_movie['trixbudget'].map(lambda x: x>0).astype(int)

In [848]:
df_movie_clean = df_movie.drop(columns=['name', 'trixbudget', 'gross', 'year'])

In [849]:
df_movie_clean.head()

,rating,genre,released,director,writer,star,country,budget,company,runtime,success
0,R,Drama,June,Stanley Kubrick,Stephen King,Jack Nicholson,United Kingdom,19000000.0,Warner Bros.,146.0,0
1,R,Adventure,July,Randal Kleiser,Henry De Vere Stacpoole,Brooke Shields,United States,4500000.0,Columbia Pictures,104.0,1
2,PG,Action,June,Irvin Kershner,Leigh Brackett,Mark Hamill,United States,18000000.0,Lucasfilm,124.0,1
3,PG,Comedy,July,Jim Abrahams,Jim Abrahams,Robert Hays,United States,3500000.0,Paramount Pictures,88.0,1
4,R,Comedy,July,Harold Ramis,Brian Doyle-Murray,Chevy Chase,United States,6000000.0,Orion Pictures,98.0,1


In [850]:
df_movie_clean.to_csv('movie_data.csv', index = False)

In [851]:
df_movie_success = df_movie.loc[df_movie['success'] == 1]

df_movie_failed = df_movie.loc[df_movie['success'] == 0]

In [852]:
director_other = df_movie_clean['director'].value_counts() < 15
df_movie_clean.loc[df_movie_clean['director'].isin(director_other.index[director_other]), 'director'] = 'others'

In [853]:
writer_other = df_movie_clean['writer'].value_counts() < 10
df_movie_clean.loc[df_movie_clean['writer'].isin(writer_other.index[writer_other]), 'writer'] = 'others'

In [854]:
star_other = df_movie_clean['star'].value_counts() < 20
df_movie_clean.loc[df_movie_clean['star'].isin(star_other.index[star_other]), 'star'] = 'others'

In [855]:
country_other = df_movie_clean['country'].value_counts() < 50
df_movie_clean.loc[df_movie_clean['country'].isin(country_other.index[country_other]), 'country'] = 'others'

In [856]:
company_other = df_movie_clean['company'].value_counts() < 150
df_movie_clean.loc[df_movie_clean['company'].isin(company_other.index[company_other]), 'company'] = 'others'

In [857]:
#rating_other = df_movie_clean['rating'].value_counts() < 100
#df_movie_clean.loc[df_movie_clean['rating'].isin(rating_other.index[rating_other]), 'rating'] = 'others'

In [858]:
df_movie_clean.head()

,rating,genre,released,director,writer,star,country,budget,company,runtime,success
0,R,Drama,June,others,Stephen King,others,United Kingdom,19000000.0,Warner Bros.,146.0,0
1,R,Adventure,July,others,others,others,United States,4500000.0,Columbia Pictures,104.0,1
2,PG,Action,June,others,others,others,United States,18000000.0,others,124.0,1
3,PG,Comedy,July,others,others,others,United States,3500000.0,Paramount Pictures,88.0,1
4,R,Comedy,July,others,others,others,United States,6000000.0,others,98.0,1


In [859]:
df_movie_clean.nunique()

rating       10
genre        15
released     12
director     22
writer       16
star         40
country       6
budget      398
company       7
runtime     127
success       2
dtype: int64

In [860]:
df_movie_dummies = pd.get_dummies(df_movie_clean)

In [861]:
df_movie_dummies.head()

,budget,runtime,success,rating_Approved,rating_G,rating_NC-17,rating_Not Rated,rating_PG,rating_PG-13,rating_R,...,country_United Kingdom,country_United States,country_others,company_Columbia Pictures,company_New Line Cinema,company_Paramount Pictures,company_Twentieth Century Fox,company_Universal Pictures,company_Warner Bros.,company_others
0,19000000.0,146.0,0,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,1,0
1,4500000.0,104.0,1,0,0,0,0,0,0,1,...,0,1,0,1,0,0,0,0,0,0
2,18000000.0,124.0,1,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,1
3,3500000.0,88.0,1,0,0,0,0,1,0,0,...,0,1,0,0,0,1,0,0,0,0
4,6000000.0,98.0,1,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,1


In [862]:
#df_movie_dummies['budget'] = df_movie_dummies['budget'].reshape(-1, 1)

In [863]:
y = df_movie_dummies["success"]

X = df_movie_dummies.drop(columns=['success'])

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [864]:
scaler = StandardScaler()

X_scaler = scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)

X_test_scaled = X_scaler.transform(X_test)

In [867]:
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=25, activation="relu", input_dim=130))
nn_model.add(tf.keras.layers.Dense(units=10, activation="relu"))
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

fit_model = nn_model.fit(X_train_scaled, y_train, epochs=25)


model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/25
128/128 [==============================] - 1s 2ms/step - loss: 0.7173 - accuracy: 0.5739
Epoch 2/25
128/128 [==============================] - 0s 2ms/step - loss: 0.6144 - accuracy: 0.6807
Epoch 3/25
128/128 [==============================] - 0s 2ms/step - loss: 0.5939 - accuracy: 0.6957
Epoch 4/25
128/128 [==============================] - 0s 2ms/step - loss: 0.5811 - accuracy: 0.7016
Epoch 5/25
128/128 [==============================] - 0s 2ms/step - loss: 0.5709 - accuracy: 0.7082
Epoch 6/25
128/128 [==============================] - 0s 3ms/step - loss: 0.5630 - accuracy: 0.7159
Epoch 7/25
128/128 [==============================] - 0s 2ms/step - loss: 0.5569 - accuracy: 0.7188
Epoch 8/25
128/128 [==============================] - 0s 2ms/step - loss: 0.5508 - accuracy: 0.7223
Epoch 9/25
128/128 [==============================] - 0s 2ms/step - loss: 0.5457 - accuracy: 0.7235
Epoch 10/25
128/128 [==============================] - 0s 2ms/step - loss: 0.5374 - accuracy: 0.7304

In [720]:
df_movie.groupby(by="director").count().sort_values(by="gross", ascending=False)["gross"].head(20)

director
Woody Allen          33
Clint Eastwood       31
Steven Spielberg     27
Ron Howard           23
Ridley Scott         23
Steven Soderbergh    23
Joel Schumacher      20
Tim Burton           19
Martin Scorsese      19
Barry Levinson       18
Spike Lee            17
Robert Zemeckis      17
Oliver Stone         17
Brian De Palma       17
Directors            16
Rob Reiner           15
Walter Hill          15
Garry Marshall       15
Renny Harlin         15
Wes Craven           15
Name: gross, dtype: int64

In [721]:
df_movie.groupby(by="writer").count().sort_values(by="gross", ascending=False)["gross"].head(100)

writer
Woody Allen        32
Stephen King       29
John Hughes        20
Luc Besson         14
Joel Coen          13
                   ..
Don Rhymer          5
Stephen Sommers     5
Steve Kloves        5
Diablo Cody         5
Wes Anderson        5
Name: gross, Length: 100, dtype: int64

In [20]:
df_movie.groupby(by="star").count().sort_values(by="gross", ascending=False)["gross"].head(100)

star
Nicolas Cage            40
Tom Hanks               38
Robert De Niro          38
Bruce Willis            34
Tom Cruise              33
                        ..
Cate Blanchett          11
Patrick Swayze          11
Burt Reynolds           11
Michael Caine           11
Joseph Gordon-Levitt    11
Name: gross, Length: 100, dtype: int64

In [21]:
df_movie_failed.groupby(by="company").count().sort_values(by="gross", ascending=False)["gross"].head(30)

company
Warner Bros.                               155
Universal Pictures                         144
Columbia Pictures                          140
Paramount Pictures                         128
Twentieth Century Fox                       74
New Line Cinema                             63
Metro-Goldwyn-Mayer (MGM)                   62
Touchstone Pictures                         49
TriStar Pictures                            45
Miramax                                     43
Walt Disney Pictures                        35
Castle Rock Entertainment                   32
Orion Pictures                              28
Dreamworks Pictures                         27
Summit Entertainment                        25
Fox 2000 Pictures                           24
Morgan Creek Entertainment                  24
Dimension Films                             22
Hollywood Pictures                          22
New Regency Productions                     21
Screen Gems                                 20
Lions

In [126]:
y = df_movie_dummies["y"]


KeyError: 'y'